In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import os

In [2]:
folder=r"C:\Users\Rajath\Desktop\Face-Mask-Detection-master\dataset\train"
cat=["mask", "no_mask"]

In [3]:
x = []
y = []

for ca in cat:
    path = os.path.join(folder, ca)
    for im in os.listdir(path):
    	im_path = os.path.join(path, im)
    	img = load_img(im_path, target_size=(224, 224))
    	img = img_to_array(img)
    	img = preprocess_input(img)

    	x.append(img)
    	y.append(ca)

lb = LabelBinarizer()
y = lb.fit_transform(y)
y = to_categorical(y)

E:\anacond3_9\lib\site-packages\PIL\Image.py:945: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


In [4]:
x = np.array(x, dtype="float32")
y = np.array(y)

(x_train, x_test, y_train, y_test) = train_test_split(x, y,
	test_size=0.20, stratify=y, random_state=42)

# construct the training image generator for data augmentation
aug = ImageDataGenerator(
	rotation_range=20,
	zoom_range=0.15,
	width_shift_range=0.2,
	height_shift_range=0.2,
	shear_range=0.15,
	horizontal_flip=True,
	fill_mode="nearest")

In [28]:
from tensorflow.keras.applications import ResNet50

In [29]:
bm = ResNet50(weights='imagenet', include_top='False',input_shape=(224,224,3))

102967424/102967424 [==============================] - 18s 0us/step


In [44]:
hm = bm.output
# hm = AveragePooling2D(pool_size=(7, 7,7,7))(hm)
# hm = Flatten(name="flatten")(hm)
hm = Dense(500, activation="relu")(hm)
hm = Dense(250, activation="relu")(hm)
hm = Dropout(0.5)(hm)
hm = Dense(128, activation="relu")(hm)
hm = Dense(64, activation="relu")(hm)
hm = Dropout(0.7)(hm)
hm = Dense(2, activation="softmax")(hm)

In [45]:
inc_model = Model(inputs=bm.input, outputs=hm)
for layer in bm.layers:
    layer.trainable = False
    
optimiser = Adam(lr=0.0001, decay=0.0001 / 20)
inc_model.compile(loss="binary_crossentropy", optimizer=optimiser,metrics=["accuracy"])
inc_model.summary()

Model: "model_7"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_4[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                            

                                                                                                  
 conv2_block3_1_relu (Activatio  (None, 56, 56, 64)  0           ['conv2_block3_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv2_block3_2_conv (Conv2D)   (None, 56, 56, 64)   36928       ['conv2_block3_1_relu[0][0]']    
                                                                                                  
 conv2_block3_2_bn (BatchNormal  (None, 56, 56, 64)  256         ['conv2_block3_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv2_block3_2_relu (Activatio  (None, 56, 56, 64)  0           ['conv2_block3_2_bn[0][0]']      
 n)       

                                                                                                  
 conv3_block3_1_relu (Activatio  (None, 28, 28, 128)  0          ['conv3_block3_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv3_block3_2_conv (Conv2D)   (None, 28, 28, 128)  147584      ['conv3_block3_1_relu[0][0]']    
                                                                                                  
 conv3_block3_2_bn (BatchNormal  (None, 28, 28, 128)  512        ['conv3_block3_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv3_block3_2_relu (Activatio  (None, 28, 28, 128)  0          ['conv3_block3_2_bn[0][0]']      
 n)       

                                                                                                  
 conv4_block2_1_bn (BatchNormal  (None, 14, 14, 256)  1024       ['conv4_block2_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv4_block2_1_relu (Activatio  (None, 14, 14, 256)  0          ['conv4_block2_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv4_block2_2_conv (Conv2D)   (None, 14, 14, 256)  590080      ['conv4_block2_1_relu[0][0]']    
                                                                                                  
 conv4_block2_2_bn (BatchNormal  (None, 14, 14, 256)  1024       ['conv4_block2_2_conv[0][0]']    
 ization) 

 conv4_block5_1_conv (Conv2D)   (None, 14, 14, 256)  262400      ['conv4_block4_out[0][0]']       
                                                                                                  
 conv4_block5_1_bn (BatchNormal  (None, 14, 14, 256)  1024       ['conv4_block5_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv4_block5_1_relu (Activatio  (None, 14, 14, 256)  0          ['conv4_block5_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv4_block5_2_conv (Conv2D)   (None, 14, 14, 256)  590080      ['conv4_block5_1_relu[0][0]']    
                                                                                                  
 conv4_blo

                                                                  'conv5_block1_3_bn[0][0]']      
                                                                                                  
 conv5_block1_out (Activation)  (None, 7, 7, 2048)   0           ['conv5_block1_add[0][0]']       
                                                                                                  
 conv5_block2_1_conv (Conv2D)   (None, 7, 7, 512)    1049088     ['conv5_block1_out[0][0]']       
                                                                                                  
 conv5_block2_1_bn (BatchNormal  (None, 7, 7, 512)   2048        ['conv5_block2_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv5_block2_1_relu (Activatio  (None, 7, 7, 512)   0           ['conv5_block2_1_bn[0][0]']      
 n)       

In [46]:
h = inc_model.fit(aug.flow(x_train, y_train, batch_size=32),steps_per_epoch=len(x_train) // 32,validation_data=(x_test, y_test),validation_steps=len(x_test) // 32,epochs=7)

Epoch 1/7
116/116 [==============================] - 365s 3s/step - loss: 0.6900 - accuracy: 0.5860 - val_loss: 0.6841 - val_accuracy: 0.5970
Epoch 2/7
116/116 [==============================] - 365s 3s/step - loss: 0.6807 - accuracy: 0.5960 - val_loss: 0.6746 - val_accuracy: 0.5970
Epoch 3/7
116/116 [==============================] - 365s 3s/step - loss: 0.6784 - accuracy: 0.5982 - val_loss: 0.6731 - val_accuracy: 0.5970
Epoch 4/7
116/116 [==============================] - 361s 3s/step - loss: 0.6759 - accuracy: 0.5963 - val_loss: 0.6706 - val_accuracy: 0.5970
Epoch 5/7
116/116 [==============================] - 376s 3s/step - loss: 0.6722 - accuracy: 0.5963 - val_loss: 0.6689 - val_accuracy: 0.5970
Epoch 6/7
116/116 [==============================] - 524s 5s/step - loss: 0.6729 - accuracy: 0.5971 - val_loss: 0.6657 - val_accuracy: 0.5970
Epoch 7/7
116/116 [==============================] - 617s 5s/step - loss: 0.6699 - accuracy: 0.5960 - val_loss: 0.6634 - val_accuracy: 0.5970
